In [3]:
import pandas as pd
import os
import numpy as np
import sc2reader

In [16]:
rep_folder = "data/s2client_blizz_replays/"
rep_names = [rep_name for root_dir, rep_dir, rep_name in os.walk(rep_folder)][0]
rep_paths = [rep_folder + rep_name for rep_name in rep_names]

In [5]:
def loadReplay(rep_path):
    replay = sc2reader.load_replay(rep_path, load_level=4)
    return replay

In [34]:
%%time
data = [loadReplay(rep_path) for rep_path in rep_paths[:200]]

CPU times: user 29 s, sys: 772 ms, total: 29.8 s
Wall time: 30.7 s


In [35]:
def getPlayersData(replay):
    player_data = [player.__dict__ for player in replay.players]
    game_length = replay.length.seconds
    return [makeDatapoint(player, game_length) for player in player_data]

In [36]:
def makeDatapoint(player, game_length):
    result = player['result']
    race = player['attribute_data']['Race']
    events = sum(1 for event in player['events'] if event.__dict__['name'] != 'CameraEvent')
    league_index = player['highest_league']
    return race, league_index, events, result, game_length, 

In [37]:
playerPairs = [getPlayersData(replay) for replay in data]
raw_dataset = [player for playerPair in playerPairs for player in playerPair]

In [38]:
cols = ['Race', 'LeagueIndex', 'Actions', 'GameResult', 'GameLength']
df = pd.DataFrame(raw_dataset, columns=cols)
df['APM'] = df['Actions'] / (df['GameLength'] / 60)
df

,Race,LeagueIndex,Actions,GameResult,GameLength,APM
0,Protoss,6,5319,Loss,932,342.424893
1,Zerg,6,5164,Win,932,332.446352
2,Terran,5,987,Win,324,182.777778
3,Terran,5,540,Loss,324,100.000000
4,Random,5,493,Loss,667,44.347826
5,Zerg,0,981,Win,667,88.245877
6,Zerg,3,1433,Win,775,110.941935
7,Terran,3,1516,Loss,775,117.367742
8,Zerg,5,1035,Win,537,115.642458
9,Zerg,5,1263,Loss,537,141.117318


In [39]:
apm_for_winners = df.groupby('GameResult').median().reset_index()[['GameResult', 'APM']]
apm_for_winners

,GameResult,APM
0,Loss,107.029071
1,Win,112.733702


In [43]:
apm_by_league = df.groupby('LeagueIndex').median().reset_index()[['LeagueIndex', 'APM']]
apm_by_league

,LeagueIndex,APM
0,0,92.243647
1,1,82.184874
2,2,68.000000
3,3,84.506293
4,4,106.954711
5,5,120.337961
6,6,199.363766
7,7,260.991662
